In [22]:
import pandas as pd
import numpy as np
import xgboost as xgb
import json
import lightgbm as lgb
import holidays
import itertools
from tqdm import tqdm

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, HuberRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
import pickle

In [23]:
def generate_oot_folds(df, date_col='date', n_folds=5, test_window=7):
    """
    Gera folds OOT com base na coluna de data, aplicável a todas as séries.
    """
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])
    unique_dates = sorted(df[date_col].unique())

    folds = []
    for i in range(n_folds):
        test_end_idx = len(unique_dates) - i * test_window
        test_start_idx = test_end_idx - test_window
        train_end_idx = test_start_idx

        if train_end_idx <= 0:
            break

        train_dates = unique_dates[:train_end_idx]
        test_dates = unique_dates[test_start_idx:test_end_idx]

        folds.append((train_dates, test_dates))

    return folds


In [24]:
def smape_metric(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0  # Evita divisão por zero
    return np.mean(diff) * 100

# Read Data
---

In [25]:
df = pd.read_parquet("./sample_trat_curva_D.parquet")

# Processing
---

## changing_names


In [26]:
renames = {
    "loja_id": "merchant_id",
    'produto_id': 'product_id',
    'data': 'date',
    'categoria_id': 'category_id',
    'is_medicamento': 'is_medicine',
    'curva': 'sales_curve',
    'estoque_final': 'ending_stock',
    'venda': 'sales',
    'custo': 'cost',
    'preco': 'price',
    'estoque_inicial': 'starting_stock',
    'estoque_final_anterior': 'previous_ending_stock',
    'reposicao': 'restock',
}

df = df.rename(columns=renames)

## droping columns

In [27]:
to_drop = [
    'ending_stock',
    'starting_stock',
    'previous_ending_stock',
    'restock',
    'sales_curve'
]

df = df.drop(columns=to_drop)

## New Features

## Date 

In [28]:
df['date'] = pd.to_datetime(df['date'])

# Extracting date features
df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
df['day_of_week'] = df['date'].dt.dayofweek
df['is_weekend'] = df['day_of_week'] >= 5
df['week_of_year'] = df['date'].dt.isocalendar().week

# Brazilian holidays
br_holidays = holidays.Brazil()
df['is_brazilian_holiday'] = df['date'].isin(br_holidays)

In [29]:
df.head()

,merchant_id,product_id,date,category_id,is_medicine,sales,cost,price,day,month,year,day_of_week,is_weekend,week_of_year,is_brazilian_holiday
19092758,1,6,2022-01-01,4.0,False,0.0,0.055587,0.07722,1,1,2022,5,True,52,False
19092759,1,6,2022-01-02,4.0,False,0.0,0.055587,0.07722,2,1,2022,6,True,52,False
19092760,1,6,2022-01-03,4.0,False,0.0,0.055587,0.07722,3,1,2022,0,False,1,False
19092761,1,6,2022-01-04,4.0,False,0.0,0.055587,0.07722,4,1,2022,1,False,1,False
19092762,1,6,2022-01-05,4.0,False,0.0,0.055587,0.07722,5,1,2022,2,False,1,False


## lags

In [30]:
# Criar colunas de semana anterior
df['prev_week'] = df['week_of_year'] - 1
df['prev_year'] = df['year']

# Ajustar quando a semana for 1 (voltar para última semana do ano anterior)
df.loc[df['week_of_year'] == 1, 'prev_week'] = 52  # ou 53 dependendo do calendário, pode ajustar se necessário
df.loc[df['week_of_year'] == 1, 'prev_year'] = df['year'] - 1

# Agrupar para calcular média de cost e price por semana
weekly_avg = df.groupby(
    ['product_id', 'merchant_id', 'year', 'week_of_year']
)[['cost', 'price', 'sales']].mean().reset_index()

# Merge com base na semana anterior
df = df.merge(
    weekly_avg,
    left_on=['product_id', 'merchant_id', 'prev_year', 'prev_week'],
    right_on=['product_id', 'merchant_id', 'year', 'week_of_year'],
    how='left',
    suffixes=('', '_prev_week')
)

# Renomear colunas de média da semana anterior
df.rename(columns={
    'cost_prev_week': 'prev_week_cost_avg',
    'sales_prev_week': 'prev_week_sales_avg',
    'price_prev_week': 'prev_week_price_avg'
}, inplace=True)

# Remover colunas auxiliares
df = df.drop(columns=[
        'year_prev_week', 
        'week_of_year_prev_week', 
        'prev_year', 
        'prev_week'
    ])

df = df.drop(columns=['cost', 'price'])

In [31]:
print(df.shape)
df = df.dropna(subset=['prev_week_cost_avg', 'prev_week_price_avg', 'prev_week_sales_avg'])
print(df.shape)

(15440443, 16)
(15335714, 16)


## Change types

In [32]:
df["date"] = pd.to_datetime(df["date"])

In [33]:
change_types = {
    'category_id': 'int64',
    'is_medicine': 'int64',
    'is_weekend': 'int64',
    'is_brazilian_holiday': 'int64',
}

df = df.astype(change_types)

In [34]:
df.head()

,merchant_id,product_id,date,category_id,is_medicine,sales,day,month,year,day_of_week,is_weekend,week_of_year,is_brazilian_holiday,prev_week_cost_avg,prev_week_price_avg,prev_week_sales_avg
0,1,6,2022-01-01,4,0,0.0,1,1,2022,5,1,52,0,0.061576,0.08712,0.0
1,1,6,2022-01-02,4,0,0.0,2,1,2022,6,1,52,0,0.061576,0.08712,0.0
9,1,6,2022-01-10,4,0,0.0,10,1,2022,0,0,2,0,0.055587,0.07722,0.0
10,1,6,2022-01-11,4,0,0.0,11,1,2022,1,0,2,0,0.055587,0.07722,0.0
11,1,6,2022-01-12,4,0,0.0,12,1,2022,2,0,2,0,0.055587,0.07722,0.0


# Train Split

In [35]:
# df["date"] = pd.to_datetime(df["date"])
# df = df.sort_values("date").reset_index(drop=True)

# split_index = int(0.8 * len(df))
# cutoff_date = df.loc[split_index, "date"]

# X = df[df["date"] <= cutoff_date].reset_index(drop=True)
# Y = df[df["date"] > cutoff_date].reset_index(drop=True)

# del df

# print(f"Tamanho do X: {len(X)}")
# print(f"Tamanho do Y: {len(Y)}")
# print(f"Data de corte: {cutoff_date}")

In [36]:
# X_train = X.drop(columns=["date", "sales"])
# Y_train = X["sales"]
# X_test = Y.drop(columns=["date", "sales"])
# Y_test = Y["sales"]

# print(X_train.shape)
# print(Y_train.shape)
# print(X_test.shape)
# print(Y_test.shape)

# Train

In [37]:
# model_params = {
#     'Ridge': {
#         'model': Ridge(),
#         'params': {
#             'alpha': [10.0, 30.0, 50.0]
#         }
#     },
#     'Lasso': {
#         'model': Lasso(),
#         'params': {
#             'alpha': [1.0, 3.0, 5.0, 10.0]
#         }
#     },
#     'DecisionTreeRegressor': {
#         'model': DecisionTreeRegressor(),
#         'params': {
#             'max_depth': [None, 10, 20, 30],
#             'min_samples_split': [2, 5, 10],
#             'min_samples_leaf': [1, 2, 4]
#         }
#     },
#     'RandomForestRegressor': {
#         'model': RandomForestRegressor(),
#         'params': {
#             'n_estimators': [20, 30]
#         }
#     },
#     'XGBRegressor': {
#         'model': xgb.XGBRegressor(),
#         'params': {
#             'n_estimators': [60, 100, 150, 200],
#             'learning_rate': [0.01, 0.1, 0.5],
#             'subsample': [0.9]
#         }
#     },
#     'LGBMRegressor': {
#         'model': LGBMRegressor(),
#         'params': {
#             'n_estimators': [40, 50, 100, 150, 200],
#             'learning_rate': [0.1, 0.2, 0.3],
#             'num_leaves': [40, 63, 70]
#         }
#     }
# }

In [38]:
model_params = {
    'Ridge': {
        'model': Ridge(),
        'params': {
            'alpha': [1.0, 5.0, 10.0] # , 30.0, 50.0
        }
    },
    'Lasso': {
        'model': Lasso(),
        'params': {
            'alpha': [0.1, 0.5, 1.0] #, 3.0, 5.0, 10.0
        }
    },
    'LGBMRegressor': {
        'model': LGBMRegressor(),
        'params': {
            'n_estimators': [100],
            'learning_rate': [0.1, 0.2, 0.3],
            'num_leaves': [40, 80]
        }
    }
}

In [39]:
# 1. Separar a última semana para validação final
max_date = df['date'].max()
val_start = max_date - pd.Timedelta(days=6)

df_val = df[df['date'] >= val_start]
df_train_full = df[df['date'] < val_start]

print(f"Validação final: de {df_val['date'].min().date()} até {df_val['date'].max().date()}")
print(f"Treino + Folds: até {df_train_full['date'].max().date()}")

Validação final: de 2025-02-15 até 2025-02-21
Treino + Folds: até 2025-02-14


In [40]:
folds = generate_oot_folds(df_train_full, date_col='date', n_folds=5, test_window=7)

# Vamos armazenar os conjuntos separados
fold_data = []

for i, (train_dates, test_dates) in enumerate(folds):
    train_df = df_train_full[df_train_full['date'].isin(train_dates)]
    test_df = df_train_full[df_train_full['date'].isin(test_dates)]

    fold_data.append({
        'fold': i + 1,
        'train': train_df,
        'test': test_df
    })

    print(f"Fold {i + 1} -> Treino: {train_df['date'].min().date()} até {train_df['date'].max().date()} | "
          f"Teste: {test_df['date'].min().date()} até {test_df['date'].max().date()}")


Fold 1 -> Treino: 2022-01-01 até 2025-02-07 | Teste: 2025-02-08 até 2025-02-14
Fold 2 -> Treino: 2022-01-01 até 2025-01-31 | Teste: 2025-02-01 até 2025-02-07
Fold 3 -> Treino: 2022-01-01 até 2025-01-24 | Teste: 2025-01-25 até 2025-01-31
Fold 4 -> Treino: 2022-01-01 até 2025-01-17 | Teste: 2025-01-18 até 2025-01-24
Fold 5 -> Treino: 2022-01-01 até 2025-01-10 | Teste: 2025-01-11 até 2025-01-17


In [41]:
import os

# Cria a pasta 'models' se ela não existir
os.makedirs('./models', exist_ok=True)

model_best_params = {}
best_preds = 0

for model_name, config in model_params.items():
    print(f"Running manual grid search for {model_name}...")

    model_class = config['model']
    param_grid = config['params']

    keys, values = zip(*param_grid.items())
    param_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

    best_score = float('inf')
    best_params = None
    best_model = None

    for params in tqdm(param_combinations):
        fold_scores = []

        for fold in fold_data:
            train = fold['train']
            test = fold['test']

            X_train = train.drop(columns=["date", "sales"])
            Y_train = train["sales"]
            X_test = test.drop(columns=["date", "sales"])
            Y_test = test["sales"]

            model = model_class.set_params(**params)
            model.fit(X_train, Y_train)

            preds = model.predict(X_test)
            preds = np.maximum(preds, 0)

            smape = smape_metric(Y_test, preds)
            fold_scores.append(smape)

        mean_smape = np.mean(fold_scores)

        if mean_smape < best_score:
            best_score = mean_smape
            best_params = params
            best_model = model
            best_preds = preds

    print(f'Best Parameters: {best_params}')
    print(f'Best SMAPE (mean across folds): {best_score:.6f}\n\n')

    model_best_params[model_name] = {
        'model': model_name,
        'best_param': best_params,
        'best_score': best_score
    }

    with open('model_best_params.json', 'w') as f:
        json.dump(model_best_params, f, indent=4)

    with open(f'./models/{model_name}.pkl', 'wb') as f:
        pickle.dump(best_model, f)

    del model, best_model, preds


Running manual grid search for Ridge...


100%|██████████| 3/3 [02:15<00:00, 45.28s/it]


Best Parameters: {'alpha': 1.0}
Best SMAPE (mean across folds): 164.454431


Running manual grid search for Lasso...


100%|██████████| 3/3 [04:41<00:00, 93.82s/it] 


Best Parameters: {'alpha': 1.0}
Best SMAPE (mean across folds): 199.906211


Running manual grid search for LGBMRegressor...


...

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.546036 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 952
[LightGBM] [Info] Number of data points in the train set: 15086847, number of used features: 13
[LightGBM] [Info] Start training from score 0.024858
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.841065 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 954
[LightGBM] [Info] Number of data points in the train set: 14963242, number of used features: 13
[LightGBM] [Info] Start training from score 0.024955
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.970211 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not e

 83%|████████▎ | 5/6 [19:41<04:41, 281.07s/it]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.571874 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 952
[LightGBM] [Info] Number of data points in the train set: 15086847, number of used features: 13
[LightGBM] [Info] Start training from score 0.024858
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.648849 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 954
[LightGBM] [Info] Number of data points in the train set: 14963242, number of used features: 13
[LightGBM] [Info] Start training from score 0.024955
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.704414 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not e

100%|██████████| 6/6 [23:01<00:00, 230.19s/it]

Best Parameters: {'n_estimators': 100, 'learning_rate': 0.3, 'num_leaves': 80}
Best SMAPE (mean across folds): 195.892886




In [43]:
# Separa X e y
X_train = df_train_full.drop(columns=["date", "sales"])
Y_train = df_train_full["sales"]
X_test = df_val.drop(columns=["date", "sales"])
Y_test = df_val["sales"]

# Ajusta modelo final
final_model = Ridge(alpha=1.0)
final_model.fit(X_train, Y_train)

# Faz predições na validação
val_preds = np.maximum(final_model.predict(X_test), 0)

# Avalia desempenho
val_mae = mean_absolute_error(Y_test, val_preds)
print(f"MAE na validação final (última semana): {val_mae:.4f}")


MAE na validação final (última semana): 0.0273
